In [9]:
from pathlib import Path
import duckdb

In [10]:
input_file = Path("../data") / "companies_sorted.parquet"

In [11]:
# Create connection
conn = duckdb.connect()

In [12]:
%%time
# Count number of rows in parquet file
tbl = conn.sql(
    f"""
    SELECT COUNT(*) AS counts FROM "{input_file}";
    """
)
counts = tbl.fetchone()[0]
print(f"{counts} rows in parquet file")

7173426 rows in parquet file
CPU times: user 3.87 ms, sys: 1.43 ms, total: 5.3 ms
Wall time: 2.42 ms


In [13]:
%%time
# Count number of rows in parquet file
tbl = conn.sql(
    f"""
    SELECT * FROM "{input_file}";
    """
)
print(tbl.columns)

['C0', 'name', 'domain', 'year founded', 'industry', 'size range', 'locality', 'country', 'linkedin url', 'current employee estimate', 'total employee estimate']
CPU times: user 422 µs, sys: 39 µs, total: 461 µs
Wall time: 461 µs


In [14]:
%%time
# Read specific columns from parquet file and sort by year founded
tbl1 = conn.sql(
    f"""
    SELECT name, country, "year founded" FROM tbl
    WHERE country IS NOT NULL AND "year founded" IS NOT NULL
    ORDER BY "year founded";
    """
)

print(f"Columns in parquet file: {tbl1.columns}")

result = tbl1.limit(10).execute()
result

Columns in parquet file: ['name', 'country', 'year founded']
CPU times: user 885 ms, sys: 42.6 ms, total: 927 ms
Wall time: 193 ms


┌────────────────────────────────────────┬────────────────┬──────────────┐
│                  name                  │    country     │ year founded │
│                varchar                 │    varchar     │    double    │
├────────────────────────────────────────┼────────────────┼──────────────┤
│ university of glasgow                  │ united kingdom │       1451.0 │
│ marine corps officer selection         │ united states  │       1775.0 │
│ us naval reserve officer               │ united states  │       1775.0 │
│ technology services organization (tso) │ united states  │       1775.0 │
│ u.s. department of the treasury        │ united states  │       1789.0 │
│ white house athletic center            │ united states  │       1789.0 │
│ old farmer's almanac                   │ united states  │       1792.0 │
│ the times herald                       │ united states  │       1799.0 │
│ us army                                │ united states  │       1800.0 │
│ leonardslee house & gar

In [15]:
%%time
# Group by industry and total number of estimated employees
tbl2 = conn.sql(
    f"""
    SELECT industry, sum("current employee estimate") AS total_employees FROM tbl
    WHERE country IS NOT NULL AND "year founded" IS NOT NULL
    GROUP BY industry
    ORDER BY total_employees DESC;
    """
)
result = tbl2.limit(10).execute()
result

CPU times: user 839 ms, sys: 26.4 ms, total: 866 ms
Wall time: 128 ms


┌─────────────────────────────────────┬─────────────────┐
│              industry               │ total_employees │
│               varchar               │     int128      │
├─────────────────────────────────────┼─────────────────┤
│ information technology and services │         5373820 │
│ higher education                    │         3171737 │
│ hospital & health care              │         2944166 │
│ financial services                  │         2697000 │
│ retail                              │         2254787 │
│ telecommunications                  │         1914927 │
│ banking                             │         1843068 │
│ oil & energy                        │         1654288 │
│ computer software                   │         1648061 │
│ insurance                           │         1392000 │
├─────────────────────────────────────┴─────────────────┤
│ 10 rows                                     2 columns │
└───────────────────────────────────────────────────────┘

In [16]:
%%time
# Group by country and total number of estimated employees
tbl3 = conn.sql(
    f"""
    SELECT country, sum("current employee estimate") AS total_employees FROM tbl
    WHERE country IS NOT NULL AND "year founded" IS NOT NULL
    GROUP BY country
    ORDER BY total_employees DESC;
    """
)
result = tbl3.limit(10).execute()
result

CPU times: user 477 ms, sys: 16.5 ms, total: 493 ms
Wall time: 75.2 ms


┌────────────────┬─────────────────┐
│    country     │ total_employees │
│    varchar     │     int128      │
├────────────────┼─────────────────┤
│ united states  │        32031261 │
│ united kingdom │         4853601 │
│ india          │         2788292 │
│ canada         │         2356054 │
│ france         │         1989108 │
│ netherlands    │         1632887 │
│ brazil         │         1518917 │
│ australia      │         1212179 │
│ germany        │         1195447 │
│ spain          │         1150026 │
├────────────────┴─────────────────┤
│ 10 rows                2 columns │
└──────────────────────────────────┘